In [1]:
import boto3
from botocore.exceptions import ClientError

dyn_resource = boto3.resource('dynamodb')

class_user_table = dyn_resource.Table("class_user-dev_cta_20221225-classu_ted")

# 하나의 테이블에서 한번에 여러개의 get_item 하는 코드
# batch_get_item은 최대 16MB, 100개의 항목에 대해서만 동작하므로 100개씩 나눠서 실행
def batch_load(class_id, user_id_list):
    class_user_list = []
    
    if not user_id_list:
        return class_user_list
    
    chunked_list = [ user_id_list[i:i+100] for i in range(0, len(user_id_list), 100) ]
    
    for i in chunked_list:
        class_user_list += batch_100_load(class_id, i)
        
    return class_user_list


def batch_100_load(class_id, user_id_list):
    batch_keys = {
        class_user_table.name: {
            "Keys": [ { "partition_key": f"class@{class_id}", "sort_key": f"user#user@{i}" } for i in user_id_list ]
        }
    }
    
    try:
        retrieved = dyn_resource.batch_get_item(RequestItems=batch_keys)
    except ClientError:
        raise
    else:
        return retrieved["Responses"][class_user_table.name]
    

class_id = "1"
user_id_list = ["1","1234799","2","3","777","Ruben","chad", "1"]
batch_data = batch_load(class_id, list(set(user_id_list)))

print(batch_data)

[{'updated_at': Decimal('1673500667425'), 'profile_image': 'https://picsum.photos/200', 'user_id': 'Ruben', 'profile_border_color': '', 'grade': Decimal('1'), 'created_at': Decimal('1673500667425'), 'nickname': '1yfgvyhvrtyvyurt', 'partition_key': 'class@1', 'total_badge_count': Decimal('0'), 'sort_key': 'user#user@Ruben', 'class_id': '1'}, {'updated_at': Decimal('1673498279907'), 'profile_image': 'https://picsum.photos/200', 'user_id': '3', 'profile_border_color': '', 'grade': Decimal('1'), 'created_at': Decimal('1673498279907'), 'nickname': 'ruben', 'partition_key': 'class@1', 'total_badge_count': Decimal('0'), 'sort_key': 'user#user@3', 'class_id': '1'}, {'updated_at': Decimal('1673499929954'), 'profile_image': 'https://picsum.photos/200', 'user_id': '777', 'profile_border_color': '', 'grade': Decimal('1'), 'created_at': Decimal('1673499929954'), 'nickname': 'kkk2', 'partition_key': 'class@1', 'total_badge_count': Decimal('0'), 'sort_key': 'user#user@777', 'class_id': '1'}, {'create